In [139]:
import pandas as pd
import numpy as np
import regex as re

In [ ]:
# import pandas as pd
# import json

# with open('ailes_avion.json') as f:
#     data = json.load(f)

# # Use pd.json_normalize to convert the JSON to a DataFrame
# df = pd.json_normalize(data)

# # Rename the columns for clarity
# df.columns = ['Name', 'Link', 'Family', 'Thickness(%)', 'x-Location_of_Max_Thk(%)', 'Camber(%)', 'Data_Sources', 'Cl_Max', 'Cl/Cd_Max', 'Cd@Cl=0.1', 'Cd@Cl=0.4', 'Cd@Cl=0.6']

# # Create a csv file
# df.to_csv("ailes_avion.csv", index=False)
# # Display the DataFrame
# df.head(5)

# Get CSV

In [140]:
pd.set_option('display.max_row', 8353 )

In [141]:
df_dat = pd.read_csv('dat_files_index.csv', usecols=('name', 'size'))
df_dat.rename(columns={'name':'Name', 'size':'Size'}, inplace=True)
print(df_dat.shape)
df_dat.head()

(8353, 2)


,Name,Size
0,a18.DAT,945
1,a18sm.DAT,1795
2,A6014-S.DAT,6116
3,A6016-S.DAT,5914
4,A6018-S.DAT,5889


In [142]:
# Checking if the dataset contains duplicates
doublon_datfile = df_dat['Name'].value_counts().index[df_dat['Name'].value_counts().values > 1]
print(f'Nombre de doublons dans la bigtable : {len(doublon_datfile)}')
print([i for i in doublon_datfile])

Nombre de doublons dans la bigtable : 0
[]


In [143]:
df_bigtable = pd.read_csv('data/ailes_avion.csv', usecols=('Name', 'Family'))
print(df_bigtable.shape)
df_bigtable.head()

(6324, 2)


,Name,Family
0,63A108 MOD C,NASA
1,A18,Uncategorized
2,A18 (SMOOTHED),Uncategorized
3,A6014-S,Ayers
4,A6016-S,Ayers


In [144]:
# Checking if the dataset contains duplicates
doublon_bigtable = df_bigtable['Name'].value_counts().index[df_bigtable['Name'].value_counts().values > 1]
print(f'Nombre de doublons dans la bigtable : {len(doublon_bigtable)}')
print([i for i in doublon_bigtable])

Nombre de doublons dans la bigtable : 2
['FX 66-17AII-182', 'BOEING 737 MIDSPAN']


# Basic Regex and merge

In [145]:
# Create a column Name_modified with all names in lowercase and without the .DAT ending 
df_dat['Name_modified'] =  df_dat['Name'].apply(lambda x : (re.split(".DAT$", str(x)))[0])
df_dat['Name_modified'] = df_dat['Name_modified'].apply(lambda x : str(x).lower())
# Create a column Name_modified with all names in lowercase
df_bigtable['Name_modified'] = df_bigtable['Name'].apply(lambda x : str(x).lower())

In [146]:
# Manual affectation of the duplicates after verification
df_bigtable.loc[596, ['Name_modified']] = 'fx6617ai'
df_bigtable.loc[597, ['Name_modified']] = 'fx6617a2'
df_bigtable.loc[154, ['Name_modified']] = 'b737c'
df_bigtable.loc[155, ['Name_modified']] = 'b737b'

In [147]:
# Count the number of family in big table
#df_bigtable['Family'].value_counts()

In [148]:
# By merging using 'left', we obtain 4905 correspondances 
df_merge = pd.merge(df_bigtable, df_dat, on='Name_modified', how='left', suffixes=('_big', '_dat'))
nb_mismatch = df_merge[df_merge['Name_dat'].isna()].shape[0]
nb_match = df_merge.shape[0] - nb_mismatch
print(f'Le dataset contient {df_merge.shape[0]} valeurs dont {nb_match} correspondent aux fichiers dat.')
print(f'Il reste {nb_mismatch} valeurs à matcher.')
df_merge.head()

Le dataset contient 6324 valeurs dont 4909 correspondent aux fichiers dat.
Il reste 1415 valeurs à matcher.


,Name_big,Family,Name_modified,Name_dat,Size
0,63A108 MOD C,NASA,63a108 mod c,NaN,NaN
1,A18,Uncategorized,a18,a18.DAT,945.0
2,A18 (SMOOTHED),Uncategorized,a18 (smoothed),NaN,NaN
3,A6014-S,Ayers,a6014-s,A6014-S.DAT,6116.0
4,A6016-S,Ayers,a6016-s,A6016-S.DAT,5914.0


# Matchmaking using families

Plus tard, il faudrait ajouter une valeur qui indique à quel point on est sûr du matchmaking

In [149]:
# Count the number of missing values per family 
family = df_merge.groupby(['Family']).count()
mask_family = (family['Name_big'] != family['Name_dat'])
family_na = family[mask_family].copy()
family_na['nb_na'] = family_na['Name_big'] - family_na['Size']
# We are interested by the 5 families with the most of na
family_na['nb_na'].sort_values(ascending=False).head(5)

Family
Gottingen        382
Eppler           201
Uncategorized    153
Wortmann         110
NASA              54
Name: nb_na, dtype: int64

### **1. Create dataframes with wings left to match**

In [150]:
# Create df_dat_left and df_big_left
df_dat_left = pd.merge(df_bigtable, df_dat, on='Name_modified', how='right', suffixes=('_big', '_dat'))
df_dat_left = df_dat_left[df_dat_left['Name_big'].isna()]
print(f"Unmatched values left in the dat folder : {df_dat_left.shape}")
df_big_left = df_merge[df_merge['Name_dat'].isna()].copy()
print(f"Unmatched values left in big table : {df_big_left.shape}")

# Create a new column for each df containing the first letter of each wing
df_dat_left['First letter'] = [x[0] for x in df_dat_left['Name_modified'].values]
df_big_left['First letter'] = [x[0] for x in df_big_left['Name_modified'].values]

# Create a new column for each df with z copy of the modified name to further work on
df_dat_left['Name_modified_by_family'] = df_dat_left['Name_modified'].copy()
df_big_left['Name_modified_by_family'] = df_big_left['Name_modified'].copy()

Unmatched values left in the dat folder : (3444, 5)
Unmatched values left in big table : (1415, 5)


In [151]:
# df_dat_left ----> ['Name_big', 'Family', 'Name_modified', 'Name_dat', 'Size', 'First letter', 'Name_modified_by_family']
# Name_big and Family are NaN
df_dat_left.drop(columns=['Name_big', 'Family'], inplace=True)
# df_big_left ----> ['Name_big', 'Family', 'Name_modified', 'Name_dat', 'Size', 'First letter', 'Name_modified_by_family']
# Name_dat and Size are NaN
df_big_left.drop(columns=['Name_dat', 'Size'], inplace=True)

### **2. Regex to match the family Yost,Eiffel,Eppeler**

- Family yost : correspondance if deleting the spaces points and / from big e
- Family eiffel : correspondance if deleting the spaces and content between parenthesis + special case for eiffel 10 (wright) - 1903 wright flyer airfoil	
- Family eppler : correspondance if replacing eppler by e and removing spaces

In [152]:
big_e = df_big_left[df_big_left['First letter'] == 'e']
dat_e = df_dat_left[df_dat_left['First letter'] == 'e']
print(f'Big table number of wings left beginning by e : {big_e.shape}')
print(f'Dat folder number of wings left beginning by e : {dat_e.shape}')
print(f'Difference : {big_e.shape[0] - dat_e.shape[0]}')

Big table number of wings left beginning by e : (221, 5)
Dat folder number of wings left beginning by e : (219, 5)
Difference : 2


In [153]:
# Family yost
big_e.loc[big_e['Family'] == 'Yost', 'Name_modified_by_family'] = big_e.loc[big_e['Family'] == 'Yost', 'Name_modified_by_family'].apply(lambda x : (re.sub("\s", "", str(x))))
big_e.loc[big_e['Family'] == 'Yost', 'Name_modified_by_family'] = big_e.loc[big_e['Family'] == 'Yost', 'Name_modified_by_family'].apply(lambda x : (re.sub("\.", "", str(x))))
big_e.loc[big_e['Family'] == 'Yost', 'Name_modified_by_family'] = big_e.loc[big_e['Family'] == 'Yost', 'Name_modified_by_family'].apply(lambda x : (re.sub("/", "", str(x))))

# Family eiffel
big_e.loc[big_e['Family'] == 'Eiffel', 'Name_modified_by_family'] = big_e.loc[big_e['Family'] == 'Eiffel', 'Name_modified_by_family'].apply(lambda x : (re.sub('\(.*?\)', "", str(x))))
big_e.loc[big_e['Family'] == 'Eiffel', 'Name_modified_by_family'] = big_e.loc[big_e['Family'] == 'Eiffel', 'Name_modified_by_family'].apply(lambda x : (re.sub("\s", "", str(x))))
big_e.loc[big_e['Family'] == 'Eiffel', 'Name_modified_by_family'] = big_e.loc[big_e['Family'] == 'Eiffel', 'Name_modified_by_family'].apply(lambda x : (re.split("-", str(x)))[0])

# Family eppler
big_e.loc[big_e['Family'] == 'Eppler', 'Name_modified_by_family'] = big_e.loc[big_e['Family'] == 'Eppler', 'Name_modified_by_family'].apply(lambda x : (re.sub("eppler", "e", str(x))))
big_e.loc[big_e['Family'] == 'Eppler', 'Name_modified_by_family'] = big_e.loc[big_e['Family'] == 'Eppler', 'Name_modified_by_family'].apply(lambda x : (re.sub("\s", "", str(x))))

In [154]:
# Incorporating the new name found (Name_modified_by_family) into the dataset with the bigtable wings left to match
df_big_left = pd.merge(df_big_left, big_e[['Name_big', 'Name_modified_by_family']], on='Name_big', how='left', suffixes=('_left', '_big'))
print(df_big_left.columns)
#df_big_left.drop(columns=['Name_modified_by_family_big'], inplace=True)
df_big_left.rename(columns={'Name_modified_by_family_left':'Name_modified_by_family_original', 'Name_modified_by_family_big':'Name_modified_by_family'}, inplace=True)

Index(['Name_big', 'Family', 'Name_modified', 'First letter',
       'Name_modified_by_family_left', 'Name_modified_by_family_big'],
      dtype='object')


In [155]:
# By merging using 'left', we obtain 4905 correspondances 
df_merge_e = pd.merge(df_big_left, df_dat_left, on='Name_modified_by_family', how='left', suffixes=('_big', '_dat'))
print(df_merge_e.shape)
nb_mismatch = df_merge_e[df_merge_e['Name_dat'].isna()].shape[0]
nb_match = df_merge_e.shape[0] - nb_mismatch
print(f'Le dataset contient {df_merge_e.shape[0]} valeurs dont {nb_match} correspondent aux fichiers dat.')
print(f'Il reste {nb_mismatch} valeurs à matcher.')
df_merge_e.head()

(1415, 10)
Le dataset contient 1415 valeurs dont 199 correspondent aux fichiers dat.
Il reste 1216 valeurs à matcher.


,Name_big,Family,Name_modified_big,First letter_big,Name_modified_by_family_original,Name_modified_by_family,Name_modified_dat,Name_dat,Size,First letter_dat
0,63A108 MOD C,NASA,63a108 mod c,6,63a108 mod c,NaN,NaN,NaN,NaN,NaN
1,A18 (SMOOTHED),Uncategorized,a18 (smoothed),a,a18 (smoothed),NaN,NaN,NaN,NaN,NaN
2,AG03 (FLAT AFT BOTTOM),Drela,ag03 (flat aft bottom),a,ag03 (flat aft bottom),NaN,NaN,NaN,NaN,NaN
3,AG44CT -02F,Drela,ag44ct -02f,a,ag44ct -02f,NaN,NaN,NaN,NaN,NaN
4,AG455CT -02F ROT.,Drela,ag455ct -02f rot.,a,ag455ct -02f rot.,NaN,NaN,NaN,NaN,NaN


### **3. Regex to match the family Gottingen**

In [156]:
# Create df_dat_left and df_big_left
df_dat_left = pd.merge(df_merge_e, df_dat_left, on='Name_modified_by_family', how='right', suffixes=('_big', '_dat'))
df_dat_left = df_dat_left[df_dat_left['Name_big'].isna()]
df_dat_left.dropna(axis=1, how='all', inplace=True)
df_dat_left = df_dat_left.set_axis([re.sub('_dat', "", str(col)) for col in df_dat_left.columns], axis=1)
print(f"Unmatched values left in the dat folder : {df_dat_left.shape}")
df_big_left = df_merge_e[df_merge_e['Name_dat'].isna()].copy()
df_big_left.dropna(axis=1, how='all', inplace=True)
df_big_left = df_big_left.set_axis([re.sub('_big', "", str(col)) for col in df_big_left.columns], axis=1)
print(f"Unmatched values left in big table : {df_big_left.shape}")

Unmatched values left in the dat folder : (3245, 5)
Unmatched values left in big table : (1216, 6)


In [157]:
big_g = df_big_left[df_big_left['First letter'] == 'g']
dat_g = df_dat_left[df_dat_left['First letter'] == 'g']
print(f'Big table number of wings left beginning by g : {big_g.shape}')
print(f'Dat folder number of wings left beginning by g : {dat_g.shape}')
print(f'Difference : {big_g.shape[0] - dat_g.shape[0]}')

Big table number of wings left beginning by g : (406, 6)
Dat folder number of wings left beginning by g : (408, 5)
Difference : -2


In [158]:
big_g = big_g.copy()

In [159]:
# Family exceptions beginning by 'g'
big_g.loc[big_g['Name_modified'] == 'gu25-5(11)8', 'Name_modified_by_family'] = 'gu255118'
big_g.loc[big_g['Name_modified'] == 'gs-1', 'Name_modified_by_family'] = 'gs1'
big_g.loc[big_g['Name_modified'] == 'griffith 30% suction airfoil', 'Name_modified_by_family'] = 'griffith30symsuction'
big_g.loc[big_g['Name_modified'] == 'goe 167 (v.karman prop.2)', 'Name_modified_by_family'] = 'goe167'
big_g.loc[big_g['Name_modified'] == 'glenn martin 2', 'Name_modified_by_family'] = 'glennmartin2'
big_g.loc[big_g['Name_modified'] == 'glenn martin 3', 'Name_modified_by_family'] = 'glennmartin3'
big_g.loc[big_g['Name_modified'] == 'glenn martin 4', 'Name_modified_by_family'] = 'glennmartin4'
# Family Gottingen
big_g.loc[big_g['Family'] == 'Gottingen', 'Name_modified_by_family'] = big_g.loc[big_g['Family'] == 'Gottingen', 'Name_modified'].apply(lambda x : (re.sub("\s", "", str(x))))
big_g.loc[big_g['Family'] == 'Gottingen', 'Name_modified_by_family'] = big_g.loc[big_g['Family'] == 'Gottingen', 'Name_modified_by_family'].apply(lambda x : (re.sub('\(.*?\)', "", str(x))))

In [160]:
big_g.columns

Index(['Name', 'Family', 'Name_modified', 'First letter',
       'Name_modified_by_family_original', 'Name_modified_by_family'],
      dtype='object')

In [161]:
df_big_left.columns

Index(['Name', 'Family', 'Name_modified', 'First letter',
       'Name_modified_by_family_original', 'Name_modified_by_family'],
      dtype='object')

In [162]:
# Incorporating the new name found (Name_modified_by_family) into the dataset with the bigtable wings left to match
df_big_left = pd.merge(df_big_left, big_g[['Name', 'Name_modified_by_family']], on='Name', how='left', suffixes=('_left', '_big'))
print(df_big_left.columns)
df_big_left.rename(columns={'Name_modified_by_family_left':'Name_modified_by_family_e', 'Name_modified_by_family_big':'Name_modified_by_family'}, inplace=True)

Index(['Name', 'Family', 'Name_modified', 'First letter',
       'Name_modified_by_family_original', 'Name_modified_by_family_left',
       'Name_modified_by_family_big'],
      dtype='object')


In [163]:
# By merging using 'inner', we obtain xx correspondances 
df_merge_g = pd.merge(df_big_left, df_dat_left, on='Name_modified_by_family', how='left', suffixes=('_big', '_dat'))
df_merge_g.dropna(axis=1, how='all', inplace=True)
print(df_merge_g.shape)
nb_mismatch = df_merge_g[df_merge_g['Name_dat'].isna()].shape[0]
nb_match = df_merge_g.shape[0] - nb_mismatch
print(f'Le dataset contient {df_merge_g.shape[0]} valeurs dont {nb_match} correspondent aux fichiers dat.')
print(f'Il reste {nb_mismatch} valeurs à matcher.')
df_merge_g.head()

(1216, 11)
Le dataset contient 1216 valeurs dont 379 correspondent aux fichiers dat.
Il reste 837 valeurs à matcher.


,Name_big,Family,Name_modified_big,First letter_big,Name_modified_by_family_original,Name_modified_by_family_e,Name_modified_by_family,Name_modified_dat,Name_dat,Size,First letter_dat
0,63A108 MOD C,NASA,63a108 mod c,6,63a108 mod c,NaN,NaN,NaN,NaN,NaN,NaN
1,A18 (SMOOTHED),Uncategorized,a18 (smoothed),a,a18 (smoothed),NaN,NaN,NaN,NaN,NaN,NaN
2,AG03 (FLAT AFT BOTTOM),Drela,ag03 (flat aft bottom),a,ag03 (flat aft bottom),NaN,NaN,NaN,NaN,NaN,NaN
3,AG44CT -02F,Drela,ag44ct -02f,a,ag44ct -02f,NaN,NaN,NaN,NaN,NaN,NaN
4,AG455CT -02F ROT.,Drela,ag455ct -02f rot.,a,ag455ct -02f rot.,NaN,NaN,NaN,NaN,NaN,NaN


### **4. Global regex with quick manual verif**

In [164]:
# Create df_dat_left and df_big_left
df_dat_left = pd.merge(df_merge_g, df_dat_left, on='Name_modified_by_family', how='right', suffixes=('_big', '_dat'))
df_dat_left = df_dat_left[df_dat_left['Name_big'].isna()]
df_dat_left.dropna(axis=1, how='all', inplace=True)
df_dat_left = df_dat_left.set_axis([re.sub('_dat', "", str(col)) for col in df_dat_left.columns], axis=1)
print(f"Unmatched values left in the dat folder : {df_dat_left.shape}")
df_big_left = df_merge_g[df_merge_g['Name_dat'].isna()].copy()
df_big_left.dropna(axis=1, how='all', inplace=True)
df_big_left = df_big_left.set_axis([re.sub('_big', "", str(col)) for col in df_big_left.columns], axis=1)
print(f"Unmatched values left in big table : {df_big_left.shape}")

Unmatched values left in the dat folder : (2866, 5)
Unmatched values left in big table : (837, 7)


In [165]:
df_big_left.columns

Index(['Name', 'Family', 'Name_modified', 'First letter',
       'Name_modified_by_family_original', 'Name_modified_by_family_e',
       'Name_modified_by_family'],
      dtype='object')

In [166]:
# remove space and - then /
df_big_left['Name_modified_final'] = df_big_left['Name_modified'].copy()
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\s", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("-", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("%", "", str(x))))

df_dat_left['Name_modified_final'] = df_dat_left['Name_modified'].copy()
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\s", "", str(x))))
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("-", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("%", "", str(x))))

df_dat_left.head()

,Name_modified_by_family,Name_modified,Name,Size,First letter,Name_modified_final
0,a18sm,a18sm,a18sm.DAT,1795,a,a18sm
1,a63a108c,a63a108c,a63a108c.DAT,2278,a,a63a108c
2,ag03,ag03,ag03.DAT,4091,a,ag03
3,ag44ct02r,ag44ct02r,ag44ct02r.DAT,3856,a,ag44ct02r
4,ag455ct02r,ag455ct02r,ag455ct02r.DAT,3859,a,ag455ct02r


In [167]:
# By merging using 'inner', we obtain xx correspondances 
df_merge_final = pd.merge(df_big_left, df_dat_left, on='Name_modified_final', how='left', suffixes=('_big', '_dat'))
df_merge_final.dropna(axis=1, how='all', inplace=True)
print(df_merge_final.shape)
nb_mismatch = df_merge_final[df_merge_final['Name_dat'].isna()].shape[0]
nb_match = df_merge_final.shape[0] - nb_mismatch
print(f'Le dataset contient {df_merge_final.shape[0]} valeurs dont {nb_match} correspondent aux fichiers dat.')
print(f'Il reste {nb_mismatch} valeurs à matcher.')
df_merge_final.head()

(837, 13)
Le dataset contient 837 valeurs dont 245 correspondent aux fichiers dat.
Il reste 592 valeurs à matcher.


,Name_big,Family,Name_modified_big,First letter_big,Name_modified_by_family_original,Name_modified_by_family_e,Name_modified_by_family_big,Name_modified_final,Name_modified_by_family_dat,Name_modified_dat,Name_dat,Size,First letter_dat
0,63A108 MOD C,NASA,63a108 mod c,6,63a108 mod c,NaN,NaN,63a108modc,NaN,NaN,NaN,NaN,NaN
1,A18 (SMOOTHED),Uncategorized,a18 (smoothed),a,a18 (smoothed),NaN,NaN,a18(smoothed),NaN,NaN,NaN,NaN,NaN
2,AG03 (FLAT AFT BOTTOM),Drela,ag03 (flat aft bottom),a,ag03 (flat aft bottom),NaN,NaN,ag03(flataftbottom),NaN,NaN,NaN,NaN,NaN
3,AG44CT -02F,Drela,ag44ct -02f,a,ag44ct -02f,NaN,NaN,ag44ct02f,NaN,NaN,NaN,NaN,NaN
4,AG455CT -02F ROT.,Drela,ag455ct -02f rot.,a,ag455ct -02f rot.,NaN,NaN,ag455ct02frot.,NaN,NaN,NaN,NaN,NaN


In [168]:
df_merge_final = df_merge_final[['Name_big', 'Family', 'Name_modified_big', 'Name_modified_final', 'Name_modified_dat', 'Name_dat']]

### **5. keep trying**

In [169]:
# Create df_dat_left and df_big_left
df_dat_left = pd.merge(df_merge_final, df_dat_left, on='Name_modified_final', how='right', suffixes=('_big', '_dat'))
df_dat_left = df_dat_left[df_dat_left['Name_big'].isna()]
df_dat_left.dropna(axis=1, how='all', inplace=True)
df_dat_left = df_dat_left.set_axis([re.sub('_dat', "", str(col)) for col in df_dat_left.columns], axis=1)
print(f"Unmatched values left in the dat folder : {df_dat_left.shape}")
df_big_left = df_merge_final[df_merge_final['Name_dat'].isna()].copy()
df_big_left.dropna(axis=1, how='all', inplace=True)
df_big_left = df_big_left.set_axis([re.sub('_big', "", str(col)) for col in df_big_left.columns], axis=1)
print(f"Unmatched values left in big table : {df_big_left.shape}")

Unmatched values left in the dat folder : (2621, 6)
Unmatched values left in big table : (592, 4)


In [170]:
# remove space and . then /
df_big_left['Name_modified_final'] = df_big_left['Name_modified'].copy()
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\s", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\.", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("/", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("-", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("%", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\(", "", str(x))))
df_big_left['Name_modified_final'] = df_big_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\)", "", str(x))))


print(df_big_left.shape)
df_dat_left['Name_modified_final'] = df_dat_left['Name_modified'].copy()
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\s", "", str(x))))
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\.", "", str(x))))
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("/", "", str(x))))
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("-", "", str(x))))
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("%", "", str(x))))
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\(", "", str(x))))
df_dat_left['Name_modified_final'] = df_dat_left.loc[:, 'Name_modified_final'].apply(lambda x : (re.sub("\)", "", str(x))))

print(df_dat_left.shape)

df_dat_left.head()

(592, 4)
(2621, 6)


,Name_modified_final,Name_modified_by_family,Name_modified,Name,Size,First letter
0,a18sm,a18sm,a18sm,a18sm.DAT,1795,a
1,a63a108c,a63a108c,a63a108c,a63a108c.DAT,2278,a
2,ag03,ag03,ag03,ag03.DAT,4091,a
3,ag44ct02r,ag44ct02r,ag44ct02r,ag44ct02r.DAT,3856,a
4,ag455ct02r,ag455ct02r,ag455ct02r,ag455ct02r.DAT,3859,a


In [171]:
df_merge_final2 = pd.merge(df_big_left, df_dat_left, on='Name_modified_final', how='left', suffixes=('_big', '_dat'))
df_merge_final2.shape

(592, 9)

In [172]:
# By merging using 'inner', we obtain xx correspondances 
df_merge_final2 = pd.merge(df_big_left, df_dat_left, on='Name_modified_final', how='left', suffixes=('_big', '_dat'))
df_merge_final2.dropna(axis=1, how='all', inplace=True)
print(df_merge_final2.shape)
nb_mismatch = df_merge_final2[df_merge_final2['Name_dat'].isna()].shape[0]
nb_match = df_merge_final2.shape[0] - nb_mismatch
print(f'Le dataset contient {df_merge_final2.shape[0]} valeurs dont {nb_match} correspondent aux fichiers dat.')
print(f'Il reste {nb_mismatch} valeurs à matcher.')
df_merge_final2.head()

(592, 9)
Le dataset contient 592 valeurs dont 86 correspondent aux fichiers dat.
Il reste 506 valeurs à matcher.


,Name_big,Family,Name_modified_big,Name_modified_final,Name_modified_by_family,Name_modified_dat,Name_dat,Size,First letter
0,63A108 MOD C,NASA,63a108 mod c,63a108modc,NaN,NaN,NaN,NaN,NaN
1,A18 (SMOOTHED),Uncategorized,a18 (smoothed),a18smoothed,NaN,NaN,NaN,NaN,NaN
2,AG03 (FLAT AFT BOTTOM),Drela,ag03 (flat aft bottom),ag03flataftbottom,NaN,NaN,NaN,NaN,NaN
3,AG44CT -02F,Drela,ag44ct -02f,ag44ct02f,NaN,NaN,NaN,NaN,NaN
4,AG455CT -02F ROT.,Drela,ag455ct -02f rot.,ag455ct02frot,NaN,NaN,NaN,NaN,NaN


In [173]:
df_merge_final2[df_merge_final2['Name_dat'].isna() ==  False].head()

,Name_big,Family,Name_modified_big,Name_modified_final,Name_modified_by_family,Name_modified_dat,Name_dat,Size,First letter
76,D.G.A. 1138,Uncategorized,d.g.a. 1138,dga1138,dga1138,dga1138,dga1138.DAT,776.0,d
102,DU 86-084/18,Delft,du 86-084/18,du8608418,du8608418,du8608418,du8608418.DAT,2304.0,d
104,E. BAMBINO 7,Uncategorized,e. bambino 7,ebambino7,ebambino7,ebambino7,ebambino7.DAT,770.0,e
135,FX 60-126/1,Wortmann,fx 60-126/1,fx601261,fx601261,fx601261,fx601261.DAT,2201.0,f
151,FX 78-K-140 A/20,Wortmann,fx 78-k-140 a/20,fx78k140a20,fx78k140a20,fx78k140a20,fx78k140a20.DAT,2215.0,f


In [174]:
# Count the number of missing values per family 
family = df_merge_final2.groupby(['Family']).count()
mask_family = (family['Name_big'] != family['Name_dat'])
family_na = family[mask_family].copy()
family_na['nb_na'] = family_na['Name_big'] - family_na['Size']
# We are interested by the 5 families with the most of na
family_na['nb_na'].sort_values(ascending=False).head(20)

Family
Uncategorized    99
Hepperle         49
Selig et. al.    48
Boeing           32
Leinauer         25
NACA/Munk        24
Wortmann         23
NASA             20
Eppler           19
Barth            15
Lockheed         14
Gulfstream       14
NACA             13
Pflug            12
Althaus          11
Gottingen         9
Drela             9
Sikorsky          8
Onera             7
Marske            6
Name: nb_na, dtype: int64

### **6. Removing parenthesis while keeping the inside content**

In [175]:
# Create df_dat_left and df_big_left
df_dat_left = pd.merge(df_merge_final2, df_dat_left, on='Name_modified_final', how='right', suffixes=('_big', '_dat'))
df_dat_left = df_dat_left[df_dat_left['Name_big'].isna()]
df_dat_left.dropna(axis=1, how='all', inplace=True)
df_dat_left = df_dat_left.set_axis([re.sub('_dat', "", str(col)) for col in df_dat_left.columns], axis=1)
print(f"Unmatched values left in the dat folder : {df_dat_left.shape}")
df_big_left = df_merge_final2[df_merge_final2['Name_dat'].isna()].copy()
df_big_left.dropna(axis=1, how='all', inplace=True)
df_big_left = df_big_left.set_axis([re.sub('_big', "", str(col)) for col in df_big_left.columns], axis=1)
print(f"Unmatched values left in big table : {df_big_left.shape}")

Unmatched values left in the dat folder : (2535, 6)
Unmatched values left in big table : (506, 4)


In [176]:
# remove space and . then /
df_big_left['Name_modified_parenthesis'] = df_big_left['Name_modified'].copy()
df_big_left['Name_modified_parenthesis'] = df_big_left.loc[:, 'Name_modified_parenthesis'].apply(lambda x : (re.sub("\(", "", str(x))))
df_big_left['Name_modified_parenthesis'] = df_big_left.loc[:, 'Name_modified_parenthesis'].apply(lambda x : (re.sub("\)", "", str(x))))


print(df_big_left.shape)
df_dat_left['Name_modified_parenthesis'] = df_dat_left['Name_modified'].copy()
df_dat_left['Name_modified_parenthesis'] = df_dat_left.loc[:, 'Name_modified_parenthesis'].apply(lambda x : (re.sub("\(", "", str(x))))
df_dat_left['Name_modified_parenthesis'] = df_dat_left.loc[:, 'Name_modified_parenthesis'].apply(lambda x : (re.sub("\)", "", str(x))))

print(df_dat_left.shape)

df_dat_left.head()

(506, 5)
(2535, 7)


,Name_modified_final,Name_modified_by_family,Name_modified,Name,Size,First letter,Name_modified_parenthesis
0,a18sm,a18sm,a18sm,a18sm.DAT,1795,a,a18sm
1,a63a108c,a63a108c,a63a108c,a63a108c.DAT,2278,a,a63a108c
2,ag03,ag03,ag03,ag03.DAT,4091,a,ag03
3,ag44ct02r,ag44ct02r,ag44ct02r,ag44ct02r.DAT,3856,a,ag44ct02r
4,ag455ct02r,ag455ct02r,ag455ct02r,ag455ct02r.DAT,3859,a,ag455ct02r


In [177]:
# By merging using 'inner', we obtain xx correspondances 
df_merge_final3 = pd.merge(df_big_left, df_dat_left, on='Name_modified_parenthesis', how='left', suffixes=('_big', '_dat'))
# df_merge_final3.dropna(axis=1, how='all', inplace=True)
print(df_merge_final3.shape)
nb_mismatch = df_merge_final3[df_merge_final3['Name_dat'].isna()].shape[0]
nb_match = df_merge_final3.shape[0] - nb_mismatch
print(f'Le dataset contient {df_merge_final3.shape[0]} valeurs dont {nb_match} correspondent aux fichiers dat.')
print(f'Il reste {nb_mismatch} valeurs à matcher.')
df_merge_final3.head()

(506, 11)
Le dataset contient 506 valeurs dont 0 correspondent aux fichiers dat.
Il reste 506 valeurs à matcher.


,Name_big,Family,Name_modified_big,Name_modified_final_big,Name_modified_parenthesis,Name_modified_final_dat,Name_modified_by_family,Name_modified_dat,Name_dat,Size,First letter
0,63A108 MOD C,NASA,63a108 mod c,63a108modc,63a108 mod c,NaN,NaN,NaN,NaN,NaN,NaN
1,A18 (SMOOTHED),Uncategorized,a18 (smoothed),a18smoothed,a18 smoothed,NaN,NaN,NaN,NaN,NaN,NaN
2,AG03 (FLAT AFT BOTTOM),Drela,ag03 (flat aft bottom),ag03flataftbottom,ag03 flat aft bottom,NaN,NaN,NaN,NaN,NaN,NaN
3,AG44CT -02F,Drela,ag44ct -02f,ag44ct02f,ag44ct -02f,NaN,NaN,NaN,NaN,NaN,NaN
4,AG455CT -02F ROT.,Drela,ag455ct -02f rot.,ag455ct02frot,ag455ct -02f rot.,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
# Count the number of missing values per family 
family = df_merge_final3.groupby(['Family']).count()
mask_family = (family['Name_big'] != family['Name_dat'])
family_na = family[mask_family].copy()
family_na['nb_na'] = family_na['Name_big'] - family_na['Size']
# We are interested by the 5 families with the most of na
family_na['nb_na'].sort_values(ascending=False).head(20)

Family
Uncategorized    99
Hepperle         49
Selig et. al.    48
Boeing           32
Leinauer         25
NACA/Munk        24
Wortmann         23
NASA             20
Eppler           19
Barth            15
Lockheed         14
Gulfstream       14
NACA             13
Pflug            12
Althaus          11
Gottingen         9
Drela             9
Sikorsky          8
Onera             7
Marske            6
Name: nb_na, dtype: int64

### **7. Removing whats between parenthesis with manual verif**

In [179]:
# Create df_dat_left and df_big_left
df_dat_left = pd.merge(df_merge_final3, df_dat_left, on='Name_modified_parenthesis', how='right', suffixes=('_big', '_dat'))
df_dat_left = df_dat_left[df_dat_left['Name_big'].isna()]
df_dat_left.dropna(axis=1, how='all', inplace=True)
df_dat_left = df_dat_left.set_axis([re.sub('_dat', "", str(col)) for col in df_dat_left.columns], axis=1)
print(f"Unmatched values left in the dat folder : {df_dat_left.shape}")
df_big_left = df_merge_final3[df_merge_final3['Name_dat'].isna()].copy()
df_big_left.dropna(axis=1, how='all', inplace=True)
df_big_left = df_big_left.set_axis([re.sub('_big', "", str(col)) for col in df_big_left.columns], axis=1)
print(f"Unmatched values left in big table : {df_big_left.shape}")

Unmatched values left in the dat folder : (2535, 7)
Unmatched values left in big table : (506, 5)


In [180]:
# remove space and . then /
df_big_left['Name_without_parenthesisl'] = df_big_left['Name_modified_final'].copy()
df_big_left['Name_without_parenthesisl'] = df_big_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub('\(.*?\)', "", str(x))))
df_big_left['parenthesis_content'] = df_big_left['Name_modified_final'].copy()
df_big_left['parenthesis_content'] = df_big_left.loc[:, 'parenthesis_content'].apply(lambda x : (re.findall('\(.*?\)', str(x))))
# df_big_left['Name_without_parenthesisl'] = df_big_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub("\.", "", str(x))))
# df_big_left['Name_without_parenthesisl'] = df_big_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub("/", "", str(x))))
# df_big_left['Name_without_parenthesisl'] = df_big_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub("-", "", str(x))))
# df_big_left['Name_without_parenthesisl'] = df_big_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub("\%", "", str(x))))
print(df_big_left.shape)
df_dat_left['Name_without_parenthesisl'] = df_dat_left['Name_modified_final'].copy()
df_dat_left['Name_without_parenthesisl'] = df_dat_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub('\(.*?\)', "", str(x))))
df_dat_left['parenthesis_content'] = df_dat_left['Name_modified_final'].copy()
df_dat_left['parenthesis_content'] = df_dat_left.loc[:, 'parenthesis_content'].apply(lambda x : (re.findall('\(.*?\)', str(x))))
# df_dat_left['Name_without_parenthesisl'] = df_dat_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub("\.", "", str(x))))
# df_dat_left['Name_without_parenthesisl'] = df_dat_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub("/", "", str(x))))
# df_dat_left['Name_without_parenthesisl'] = df_dat_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub("-", "", str(x))))
# df_dat_left['Name_without_parenthesisl'] = df_dat_left.loc[:, 'Name_without_parenthesisl'].apply(lambda x : (re.sub("%", "", str(x))))
print(df_dat_left.shape)

df_dat_left.head()

(506, 7)
(2535, 9)


,Name_modified_parenthesis,Name_modified_final,Name_modified_by_family,Name_modified,Name,Size,First letter,Name_without_parenthesisl,parenthesis_content
0,a18sm,a18sm,a18sm,a18sm,a18sm.DAT,1795,a,a18sm,[]
1,a63a108c,a63a108c,a63a108c,a63a108c,a63a108c.DAT,2278,a,a63a108c,[]
2,ag03,ag03,ag03,ag03,ag03.DAT,4091,a,ag03,[]
3,ag44ct02r,ag44ct02r,ag44ct02r,ag44ct02r,ag44ct02r.DAT,3856,a,ag44ct02r,[]
4,ag455ct02r,ag455ct02r,ag455ct02r,ag455ct02r,ag455ct02r.DAT,3859,a,ag455ct02r,[]


Traitement des doublons

In [181]:
# Vérification des doublons après le retrait du contenu entre parenthèses
print('Nombre de doublon dans le dossier dat : ', df_dat_left[df_dat_left['Name_without_parenthesisl'].value_counts().values != 1].count().sum())
# Aucun doublon pour les ailes du fichier dat
doublon_big = df_big_left['Name_without_parenthesisl'].value_counts().index[df_big_left['Name_without_parenthesisl'].value_counts().values > 1]
print(f'Nombre de doublons dans la bigtable : {len(doublon_big)}')
print([i for i in doublon_big])

Nombre de doublon dans le dossier dat :  0
Nombre de doublons dans la bigtable : 1
['sikorskysc1094r8']


In [182]:
doublon_index = []
for i in doublon_big:
    temp = df_big_left[df_big_left['Name_without_parenthesisl'] == i].index
    doublon_index.append(temp[0])
    doublon_index.append(temp[1])
print(doublon_big.values)
print(doublon_index)

['sikorskysc1094r8']
[478, 479]


In [183]:
df_big_left.loc[doublon_index, :]

,Name,Family,Name_modified,Name_modified_final,Name_modified_parenthesis,Name_without_parenthesisl,parenthesis_content
478,SIKORSKY SC1094 R8,Sikorsky,sikorsky sc1094 r8,sikorskysc1094r8,sikorsky sc1094 r8,sikorskysc1094r8,[]
479,SIKORSKY SC1094R8,Sikorsky,sikorsky sc1094r8,sikorskysc1094r8,sikorsky sc1094r8,sikorskysc1094r8,[]


In [184]:
# Manual affectation of the duplicates after verification
# à vérifier
# df_big_left.loc[402, ['Name_without_parenthesisl']] = 'm6_65'
# df_big_left.loc[403, ['Name_without_parenthesisl']] = 'm6_85'
# df_big_left.loc[450, ['Name_without_parenthesisl']] = 'rc410'
# df_big_left.loc[451, ['Name_without_parenthesisl']] = 'rc510'

In [185]:
df_merge_final4 = pd.merge(df_big_left, df_dat_left, on='Name_without_parenthesisl', how='left', suffixes=('_big', '_dat'))
df_merge_final4.shape

(506, 15)

In [186]:
# Count the number of missing values per family 
family = df_merge_final4.groupby(['Family']).count()
mask_family = (family['Name_big'] != family['Name_dat'])
family_na = family[mask_family].copy()
family_na['nb_na'] = family_na['Name_big'] - family_na['Size']
# We are interested by the 5 families with the most of na
family_na['nb_na'].sort_values(ascending=False).sum()

506

In [187]:
# faire des fonctions
